# DIME Implementation: PRF and LLM-based Dimension Importance Estimation

This notebook implements the two main approaches from the DIME paper:
1. **PRF-based approach**: Uses Pseudo-Relevance Feedback to compute centroids and determine dimension importance
2. **LLM-based approach**: Uses LLM-generated documents to determine dimension importance

Each approach supports both:
- **Rerank**: Re-score existing retrieval results with modified query vectors
- **Refetch**: Perform new retrieval with modified query vectors

In [1]:
# Imports
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import json
import os
import random
from typing import List, Dict, Tuple, Optional

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

/home/krishnamursw/personal/dime/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully!


In [2]:
# Load data and model
df = pd.read_csv('/home/krishnamursw/personal/dime/apparel_dataset.csv')
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all products
print("Generating embeddings...")
product_embeddings = model.encode(
    df['text'].tolist(), 
    batch_size=16, 
    show_progress_bar=True,
    normalize_embeddings=True  # Normalize for dot product similarity
)

print(f"Product embeddings shape: {product_embeddings.shape}")
print(f"Sample embedding: {product_embeddings[0][:5]}")  # First 5 values

Generating embeddings...


Batches: 100%|██████████| 125/125 [00:24<00:00,  5.10it/s]

Product embeddings shape: (2000, 384)
Sample embedding: [-0.03355468 -0.01013993  0.05335136 -0.03003556  0.01181867]


In [12]:
# Utility functions from query.py
def normalize_scores(arr, t_min=0, t_max=1):
    """Normalize array values to [t_min, t_max] range"""
    diff = t_max - t_min
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    if arr_max == arr_min:
        return np.full_like(arr, (t_min + t_max) / 2)
    norm_arr = (((arr - arr_min) * diff) / (arr_max - arr_min)) + t_min
    return norm_arr

def softmax(scores, temperature=1.0):
    """Compute softmax over an array of scores with optional temperature"""
    scaled_scores = scores / temperature
    exps = np.exp(scaled_scores - np.max(scaled_scores))
    return exps / np.sum(exps)

def compute_centroid(embeddings, scores, k=None, weighted=False, attention_type="linear", temperature=1.0):
    """
    Compute the centroid of the top-k embeddings with optional weighting
    
    Args:
        embeddings: Array of embeddings
        scores: Array of scores corresponding to embeddings
        k: Number of top embeddings to use (None = use all)
        weighted: Whether to use weighted average
        attention_type: Type of attention ("linear" or "softmax")
        temperature: Temperature for softmax attention
        
    Returns:
        Centroid vector
    """
    if k is not None and k > 0:
        # Sort by scores (descending) and take top-k
        sorted_indices = np.argsort(scores)[::-1][:k]
        embeddings = embeddings[sorted_indices]
        scores = scores[sorted_indices]
    
    if weighted:
        if attention_type == "linear":
            weights = normalize_scores(scores, 0, 1)
        else:  # "softmax"
            weights = softmax(scores, temperature)
        centroid = np.average(embeddings, axis=0, weights=weights)
    else:
        centroid = np.mean(embeddings, axis=0)
    
    return centroid

def zero_out_least_important_dims(centroid, query_vector, alpha=0.8):
    """
    Zero out the lowest (1-alpha) fraction of dimensions in query_vector
    based on element-wise product between centroid and query_vector
    
    Args:
        centroid: Centroid vector for dimension importance
        query_vector: Original query vector
        alpha: Fraction of dimensions to keep (0.8 means keep 80%)
        
    Returns:
        Modified query vector with least important dimensions zeroed out
    """
    # Element-wise product to determine dimension importance
    importance_vec = np.multiply(centroid, query_vector)
    dim = len(importance_vec)
    keep_count = int(alpha * dim)
    
    if keep_count >= dim:
        return query_vector.copy()
    
    # Get indices of most important dimensions
    sorted_indices = np.argsort(importance_vec)
    keep_indices = set(sorted_indices[-keep_count:])
    
    # Zero out least important dimensions
    modified_query_vector = query_vector.copy()
    for i in range(dim):
        if i not in keep_indices:
            modified_query_vector[i] = 0.0
    
    return modified_query_vector

def basic_search(query, top_k=10):
    """Basic search function for initial retrieval"""
    query_embedding = model.encode([query], normalize_embeddings=True)[0]
    scores = np.dot(product_embeddings, query_embedding)
    top_indices = np.argsort(scores)[::-1][:top_k]
    
    results = []
    for i, idx in enumerate(top_indices):
        results.append({
            'rank': i + 1,
            'index': idx,
            'product_id': df.iloc[idx]['product_id'],
            'title': df.iloc[idx]['title'],
            'score': scores[idx],
            'embedding': product_embeddings[idx]
        })
    
    return results, query_embedding

print("Utility functions defined successfully!")

Utility functions defined successfully!


## 1. PRF-based Approach

This approach uses Pseudo-Relevance Feedback to compute centroids from initially retrieved documents and determine dimension importance.

In [6]:
class PRFBasedDIME:
    """
    PRF-based Dimension Importance Estimation for Dense Retrieval
    """
    
    def __init__(self, model, embeddings, dataframe):
        self.model = model
        self.embeddings = embeddings
        self.df = dataframe
        
    def prf_rerank(self, query, initial_top_k=1000, prf_k=10, final_top_k=10, 
                   zero_out_ratio=0.2, weighted=False, attention_type="linear", temperature=1.0):
        """
        PRF-based reranking: modify query vector and re-score existing results
        
        Args:
            query: Search query text
            initial_top_k: Number of documents to retrieve initially
            prf_k: Number of top documents to use for PRF
            final_top_k: Number of final results to return
            zero_out_ratio: Fraction of dimensions to zero out (0.2 means zero out 20%)
            weighted: Whether to use weighted centroid
            attention_type: Type of attention weighting
            temperature: Temperature for softmax
            
        Returns:
            Dictionary with original and reranked results
        """
        # Step 1: Initial retrieval
        print(f"Step 1: Initial retrieval (top-{initial_top_k})...")
        initial_results, original_query_embedding = basic_search(query, initial_top_k)
        
        # Step 2: Compute centroid from top-k PRF documents
        print(f"Step 2: Computing centroid from top-{prf_k} PRF documents...")
        prf_embeddings = np.array([r['embedding'] for r in initial_results[:prf_k]])
        prf_scores = np.array([r['score'] for r in initial_results[:prf_k]])
        
        centroid = compute_centroid(prf_embeddings, prf_scores, k=None, 
                                  weighted=weighted, attention_type=attention_type, 
                                  temperature=temperature)
        
        # Step 3: Zero out least important dimensions
        print(f"Step 3: Zeroing out {zero_out_ratio*100}% least important dimensions...")
        alpha = 1 - zero_out_ratio
        modified_query_embedding = zero_out_least_important_dims(centroid, original_query_embedding, alpha)
        
        # Step 4: Re-score all initial results with modified query
        print(f"Step 4: Re-scoring with modified query...")
        reranked_results = []
        for result in initial_results:
            new_score = float(np.dot(modified_query_embedding, result['embedding']))
            reranked_results.append({
                'rank': result['rank'],
                'index': result['index'],
                'product_id': result['product_id'],
                'title': result['title'],
                'original_score': result['score'],
                'new_score': new_score
            })
        
        # Sort by new scores and take top-k
        reranked_results.sort(key=lambda x: -x['new_score'])
        final_results = reranked_results[:final_top_k]
        
        # Update ranks
        for i, result in enumerate(final_results):
            result['rank'] = i + 1
        
        return {
            'method': 'prf_rerank',
            'query': query,
            'prf_k': prf_k,
            'zero_out_ratio': zero_out_ratio,
            'weighted': weighted,
            'attention_type': attention_type,
            'original_results': initial_results[:final_top_k],
            'reranked_results': final_results,
            'centroid': centroid,
            'original_query_embedding': original_query_embedding,
            'modified_query_embedding': modified_query_embedding
        }
    
    def prf_refetch(self, query, initial_top_k=1000, prf_k=10, final_top_k=10, 
                    zero_out_ratio=0.2, weighted=False, attention_type="linear", temperature=1.0):
        """
        PRF-based refetching: modify query vector and perform new retrieval
        
        Args:
            query: Search query text
            initial_top_k: Number of documents to retrieve initially for PRF
            prf_k: Number of top documents to use for PRF
            final_top_k: Number of final results to return after refetch
            zero_out_ratio: Fraction of dimensions to zero out
            weighted: Whether to use weighted centroid
            attention_type: Type of attention weighting
            temperature: Temperature for softmax
            
        Returns:
            Dictionary with original and refetched results
        """
        # Step 1: Initial retrieval for PRF
        print(f"Step 1: Initial retrieval for PRF (top-{initial_top_k})...")
        initial_results, original_query_embedding = basic_search(query, initial_top_k)
        
        # Step 2: Compute centroid from top-k PRF documents
        print(f"Step 2: Computing centroid from top-{prf_k} PRF documents...")
        prf_embeddings = np.array([r['embedding'] for r in initial_results[:prf_k]])
        prf_scores = np.array([r['score'] for r in initial_results[:prf_k]])
        
        centroid = compute_centroid(prf_embeddings, prf_scores, k=None, 
                                  weighted=weighted, attention_type=attention_type, 
                                  temperature=temperature)
        
        # Step 3: Zero out least important dimensions
        print(f"Step 3: Zeroing out {zero_out_ratio*100}% least important dimensions...")
        alpha = 1 - zero_out_ratio
        modified_query_embedding = zero_out_least_important_dims(centroid, original_query_embedding, alpha)
        
        # Step 4: Perform new retrieval with modified query
        print(f"Step 4: Performing new retrieval with modified query...")
        refetch_scores = np.dot(self.embeddings, modified_query_embedding)
        top_indices = np.argsort(refetch_scores)[::-1][:final_top_k]
        
        refetched_results = []
        for i, idx in enumerate(top_indices):
            refetched_results.append({
                'rank': i + 1,
                'index': idx,
                'product_id': self.df.iloc[idx]['product_id'],
                'title': self.df.iloc[idx]['title'],
                'score': refetch_scores[idx]
            })
        
        return {
            'method': 'prf_refetch',
            'query': query,
            'prf_k': prf_k,
            'zero_out_ratio': zero_out_ratio,
            'weighted': weighted,
            'attention_type': attention_type,
            'original_results': initial_results[:final_top_k],
            'refetched_results': refetched_results,
            'centroid': centroid,
            'original_query_embedding': original_query_embedding,
            'modified_query_embedding': modified_query_embedding
        }

# Initialize PRF-based DIME
prf_dime = PRFBasedDIME(model, product_embeddings, df)
print("PRF-based DIME initialized successfully!")

PRF-based DIME initialized successfully!


## 2. LLM-based Approach

This approach uses LLM-generated documents to determine dimension importance directly without requiring initial retrieval.

In [13]:
class LLMBasedDIME:
    """
    LLM-based Dimension Importance Estimation for Dense Retrieval
    """
    
    def __init__(self, model, embeddings, dataframe):
        self.model = model
        self.embeddings = embeddings
        self.df = dataframe
        
    def generate_llm_doc(self, query):
        """
        Generate LLM-style document for the query
        Since we don't have actual LLM, we'll simulate by creating expanded query
        """
        # Simple expansion - in real implementation, this would use an LLM
        expanded_queries = {
            "black dress shirt": "elegant black dress shirt formal business professional men's clothing cotton long sleeve button up office wear",
            "blue jeans": "blue denim jeans casual wear pants trousers men women comfortable cotton everyday fashion",
            "white sneakers": "white athletic sneakers shoes casual sports footwear comfortable rubber sole walking running",
            "red dress": "red dress women's formal elegant party evening wear special occasion outfit stylish",
            "brown leather boots": "brown leather boots shoes footwear men's outdoor durable sturdy walking hiking",
            "casual t-shirt": "casual t-shirt comfortable cotton everyday wear men women basic wardrobe essential",
            "winter coat": "winter coat warm jacket outerwear cold weather protection insulated heavy duty",
            "summer dress": "summer dress light breathable women's warm weather casual comfortable seasonal clothing"
        }
        
        # Find best match or use original query
        best_match = query.lower()
        for key in expanded_queries:
            if key in query.lower():
                best_match = key
                break
        
        return expanded_queries.get(best_match, f"clothing apparel fashion {query} style comfortable quality")
    
    def llm_rerank(self, query, initial_top_k=1000, final_top_k=10, zero_out_ratio=0.2):
        """
        LLM-based reranking: use LLM-generated doc to modify query vector and re-score
        
        Args:
            query: Search query text
            initial_top_k: Number of documents to retrieve initially
            final_top_k: Number of final results to return
            zero_out_ratio: Fraction of dimensions to zero out
            
        Returns:
            Dictionary with original and reranked results
        """
        # Step 1: Initial retrieval
        print(f"Step 1: Initial retrieval (top-{initial_top_k})...")
        initial_results, original_query_embedding = basic_search(query, initial_top_k)
        
        # Step 2: Generate LLM document and embed it
        print(f"Step 2: Generating and embedding LLM document...")
        llm_doc = self.generate_llm_doc(query)
        print(f"Generated LLM doc: {llm_doc}")
        llm_embedding = self.model.encode([llm_doc], normalize_embeddings=True)[0]
        
        # Step 3: Zero out least important dimensions using LLM embedding
        print(f"Step 3: Zeroing out {zero_out_ratio*100}% least important dimensions...")
        alpha = 1 - zero_out_ratio
        modified_query_embedding = zero_out_least_important_dims(llm_embedding, original_query_embedding, alpha)
        
        # Step 4: Re-score all initial results with modified query
        print(f"Step 4: Re-scoring with modified query...")
        reranked_results = []
        for result in initial_results:
            new_score = float(np.dot(modified_query_embedding, result['embedding']))
            reranked_results.append({
                'rank': result['rank'],
                'index': result['index'],
                'product_id': result['product_id'],
                'title': result['title'],
                'original_score': result['score'],
                'new_score': new_score
            })
        
        # Sort by new scores and take top-k
        reranked_results.sort(key=lambda x: -x['new_score'])
        final_results = reranked_results[:final_top_k]
        
        # Update ranks
        for i, result in enumerate(final_results):
            result['rank'] = i + 1
        
        return {
            'method': 'llm_rerank',
            'query': query,
            'llm_doc': llm_doc,
            'zero_out_ratio': zero_out_ratio,
            'original_results': initial_results[:final_top_k],
            'reranked_results': final_results,
            'llm_embedding': llm_embedding,
            'original_query_embedding': original_query_embedding,
            'modified_query_embedding': modified_query_embedding
        }
    
    def llm_refetch(self, query, final_top_k=10, zero_out_ratio=0.2):
        """
        LLM-based refetching: use LLM-generated doc to modify query vector and perform new retrieval
        
        Args:
            query: Search query text
            final_top_k: Number of final results to return after refetch
            zero_out_ratio: Fraction of dimensions to zero out
            
        Returns:
            Dictionary with original and refetched results
        """
        # Step 1: Get original query embedding and initial results for comparison
        print(f"Step 1: Getting original query embedding...")
        original_results, original_query_embedding = basic_search(query, final_top_k)
        
        # Step 2: Generate LLM document and embed it
        print(f"Step 2: Generating and embedding LLM document...")
        llm_doc = self.generate_llm_doc(query)
        print(f"Generated LLM doc: {llm_doc}")
        llm_embedding = self.model.encode([llm_doc], normalize_embeddings=True)[0]
        
        # Step 3: Zero out least important dimensions using LLM embedding
        print(f"Step 3: Zeroing out {zero_out_ratio*100}% least important dimensions...")
        alpha = 1 - zero_out_ratio
        modified_query_embedding = zero_out_least_important_dims(llm_embedding, original_query_embedding, alpha)
        
        # Step 4: Perform new retrieval with modified query
        print(f"Step 4: Performing new retrieval with modified query...")
        refetch_scores = np.dot(self.embeddings, modified_query_embedding)
        top_indices = np.argsort(refetch_scores)[::-1][:final_top_k]
        
        refetched_results = []
        for i, idx in enumerate(top_indices):
            refetched_results.append({
                'rank': i + 1,
                'index': idx,
                'product_id': self.df.iloc[idx]['product_id'],
                'title': self.df.iloc[idx]['title'],
                'score': refetch_scores[idx]
            })
        
        return {
            'method': 'llm_refetch',
            'query': query,
            'llm_doc': llm_doc,
            'zero_out_ratio': zero_out_ratio,
            'original_results': original_results,
            'refetched_results': refetched_results,
            'llm_embedding': llm_embedding,
            'original_query_embedding': original_query_embedding,
            'modified_query_embedding': modified_query_embedding
        }

# Initialize LLM-based DIME
llm_dime = LLMBasedDIME(model, product_embeddings, df)
print("LLM-based DIME initialized successfully!")

LLM-based DIME initialized successfully!


## 2.5. Magnitude-based Approach

This approach uses the magnitude (absolute value) of query dimensions to determine importance. The assumption is that larger magnitude dimensions are more important, while smaller magnitude dimensions represent noise or less relevant information.

In [ ]:
class MagnitudeBasedDIME:
    """
    Magnitude-based Dimension Importance Estimation for Dense Retrieval
    Uses |qi| to determine dimension importance
    """
    
    def __init__(self, model, embeddings, dataframe):
        self.model = model
        self.embeddings = embeddings
        self.df = dataframe
        
    def magnitude_rerank(self, query, initial_top_k=1000, final_top_k=10, zero_out_ratio=0.2):
        """
        Magnitude-based reranking: use query magnitude to determine importance and re-score
        
        Args:
            query: Search query text
            initial_top_k: Number of documents to retrieve initially
            final_top_k: Number of final results to return
            zero_out_ratio: Fraction of dimensions to zero out
            
        Returns:
            Dictionary with original and reranked results
        """
        # Step 1: Initial retrieval
        print(f"Step 1: Initial retrieval (top-{initial_top_k})...")
        initial_results, original_query_embedding = basic_search(query, initial_top_k)
        
        # Step 2: Compute dimension importance using magnitude
        print(f"Step 2: Computing dimension importance using magnitude...")
        importance_scores = np.abs(original_query_embedding)
        
        # Step 3: Zero out least important dimensions based on magnitude
        print(f"Step 3: Zeroing out {zero_out_ratio*100}% least important dimensions...")
        alpha = 1 - zero_out_ratio
        modified_query_embedding = self._zero_out_by_magnitude(original_query_embedding, importance_scores, alpha)
        
        # Step 4: Re-score all initial results with modified query
        print(f"Step 4: Re-scoring with modified query...")
        reranked_results = []
        for result in initial_results:
            new_score = float(np.dot(modified_query_embedding, result['embedding']))
            reranked_results.append({
                'rank': result['rank'],
                'index': result['index'],
                'product_id': result['product_id'],
                'title': result['title'],
                'original_score': result['score'],
                'new_score': new_score
            })
        
        # Sort by new scores and take top-k
        reranked_results.sort(key=lambda x: -x['new_score'])
        final_results = reranked_results[:final_top_k]
        
        # Update ranks
        for i, result in enumerate(final_results):
            result['rank'] = i + 1
        
        return {
            'method': 'magnitude_rerank',
            'query': query,
            'zero_out_ratio': zero_out_ratio,
            'original_results': initial_results[:final_top_k],
            'reranked_results': final_results,
            'importance_scores': importance_scores,
            'original_query_embedding': original_query_embedding,
            'modified_query_embedding': modified_query_embedding
        }
    
    def magnitude_refetch(self, query, final_top_k=10, zero_out_ratio=0.2):
        """
        Magnitude-based refetching: use query magnitude to determine importance and perform new retrieval
        
        Args:
            query: Search query text
            final_top_k: Number of final results to return after refetch
            zero_out_ratio: Fraction of dimensions to zero out
            
        Returns:
            Dictionary with original and refetched results
        """
        # Step 1: Get original query embedding and initial results for comparison
        print(f"Step 1: Getting original query embedding...")
        original_results, original_query_embedding = basic_search(query, final_top_k)
        
        # Step 2: Compute dimension importance using magnitude
        print(f"Step 2: Computing dimension importance using magnitude...")
        importance_scores = np.abs(original_query_embedding)
        
        # Step 3: Zero out least important dimensions based on magnitude
        print(f"Step 3: Zeroing out {zero_out_ratio*100}% least important dimensions...")
        alpha = 1 - zero_out_ratio
        modified_query_embedding = self._zero_out_by_magnitude(original_query_embedding, importance_scores, alpha)
        
        # Step 4: Perform new retrieval with modified query
        print(f"Step 4: Performing new retrieval with modified query...")
        refetch_scores = np.dot(self.embeddings, modified_query_embedding)
        top_indices = np.argsort(refetch_scores)[::-1][:final_top_k]
        
        refetched_results = []
        for i, idx in enumerate(top_indices):
            refetched_results.append({
                'rank': i + 1,
                'index': idx,
                'product_id': self.df.iloc[idx]['product_id'],
                'title': self.df.iloc[idx]['title'],
                'score': refetch_scores[idx]
            })
        
        return {
            'method': 'magnitude_refetch',
            'query': query,
            'zero_out_ratio': zero_out_ratio,
            'original_results': original_results,
            'refetched_results': refetched_results,
            'importance_scores': importance_scores,
            'original_query_embedding': original_query_embedding,
            'modified_query_embedding': modified_query_embedding
        }
    
    def _zero_out_by_magnitude(self, query_vector, importance_scores, alpha=0.8):
        """
        Zero out the lowest (1-alpha) fraction of dimensions based on magnitude importance
        
        Args:
            query_vector: Original query vector
            importance_scores: Magnitude-based importance scores (|qi|)
            alpha: Fraction of dimensions to keep (0.8 means keep 80%)
            
        Returns:
            Modified query vector with least important dimensions zeroed out
        """
        dim = len(importance_scores)
        keep_count = int(alpha * dim)
        
        if keep_count >= dim:
            return query_vector.copy()
        
        # Get indices of most important dimensions (highest magnitude)
        sorted_indices = np.argsort(importance_scores)
        keep_indices = set(sorted_indices[-keep_count:])
        
        # Zero out least important dimensions
        modified_query_vector = query_vector.copy()
        for i in range(dim):
            if i not in keep_indices:
                modified_query_vector[i] = 0.0
        
        return modified_query_vector

# Initialize Magnitude-based DIME
magnitude_dime = MagnitudeBasedDIME(model, product_embeddings, df)
print("Magnitude-based DIME initialized successfully!")

## 3. Testing and Comparison

Let's test all six approaches (PRF rerank, PRF refetch, LLM rerank, LLM refetch, Magnitude rerank, Magnitude refetch) with sample queries.

In [ ]:
def test_all_approaches(query, prf_k=5, zero_out_ratio=0.2, final_top_k=5):
    """
    Test all six approaches for a given query
    """
    print(f"\n{'='*60}")
    print(f"Testing Query: '{query}'")
    print(f"{'='*60}")
    
    results = {}
    
    # 1. PRF Rerank
    print(f"\n1. PRF RERANK:")
    results['prf_rerank'] = prf_dime.prf_rerank(
        query, prf_k=prf_k, zero_out_ratio=zero_out_ratio, final_top_k=final_top_k
    )
    
    # 2. PRF Refetch
    print(f"\n2. PRF REFETCH:")
    results['prf_refetch'] = prf_dime.prf_refetch(
        query, prf_k=prf_k, zero_out_ratio=zero_out_ratio, final_top_k=final_top_k
    )
    
    # 3. LLM Rerank
    print(f"\n3. LLM RERANK:")
    results['llm_rerank'] = llm_dime.llm_rerank(
        query, zero_out_ratio=zero_out_ratio, final_top_k=final_top_k
    )
    
    # 4. LLM Refetch
    print(f"\n4. LLM REFETCH:")
    results['llm_refetch'] = llm_dime.llm_refetch(
        query, zero_out_ratio=zero_out_ratio, final_top_k=final_top_k
    )
    
    # 5. Magnitude Rerank
    print(f"\n5. MAGNITUDE RERANK:")
    results['magnitude_rerank'] = magnitude_dime.magnitude_rerank(
        query, zero_out_ratio=zero_out_ratio, final_top_k=final_top_k
    )
    
    # 6. Magnitude Refetch
    print(f"\n6. MAGNITUDE REFETCH:")
    results['magnitude_refetch'] = magnitude_dime.magnitude_refetch(
        query, zero_out_ratio=zero_out_ratio, final_top_k=final_top_k
    )
    
    return results

def display_results(results):
    """
    Display results for all approaches
    """
    query = list(results.values())[0]['query']
    
    print(f"\n{'='*80}")
    print(f"RESULTS COMPARISON FOR: '{query}'")
    print(f"{'='*80}")
    
    # Get baseline results (original)
    baseline_results = results['prf_rerank']['original_results']
    print(f"\nBASELINE (Original Dense Retrieval):")
    print("-" * 50)
    for i, result in enumerate(baseline_results[:5], 1):
        print(f"{i}. {result['title']} - Score: {result['score']:.4f}")
    
    # Show results for each approach
    for method_name, method_results in results.items():
        print(f"\n{method_name.upper().replace('_', ' ')}:")
        print("-" * 50)
        
        # Get the appropriate results key
        if 'reranked_results' in method_results:
            final_results = method_results['reranked_results']
            score_key = 'new_score'
        else:
            final_results = method_results['refetched_results']
            score_key = 'score'
        
        for i, result in enumerate(final_results[:5], 1):
            score = result[score_key]
            print(f"{i}. {result['title']} - Score: {score:.4f}")
        
        # Show LLM doc if available
        if 'llm_doc' in method_results:
            print(f"   LLM Doc: {method_results['llm_doc'][:100]}...")

# Test with sample queries
test_queries = [
    "black dress shirt",
    "blue jeans",
    "white sneakers",
    "red dress",
    "casual t-shirt",
    "winter coat",
    "summer dress",
]

# Run tests
all_results = {}
for query in test_queries:  # Test first 2 queries
    all_results[query] = test_all_approaches(query)
    display_results(all_results[query])


Testing Query: 'black dress shirt'

1. PRF RERANK:
Step 1: Initial retrieval (top-1000)...
Step 2: Computing centroid from top-5 PRF documents...
Step 3: Zeroing out 20.0% least important dimensions...
Step 4: Re-scoring with modified query...

2. PRF REFETCH:
Step 1: Initial retrieval for PRF (top-1000)...
Step 2: Computing centroid from top-5 PRF documents...
Step 3: Zeroing out 20.0% least important dimensions...
Step 4: Performing new retrieval with modified query...

3. LLM RERANK:
Step 1: Initial retrieval (top-1000)...
Step 2: Computing centroid from top-5 PRF documents...
Step 3: Zeroing out 20.0% least important dimensions...
Step 4: Re-scoring with modified query...

2. PRF REFETCH:
Step 1: Initial retrieval for PRF (top-1000)...
Step 2: Computing centroid from top-5 PRF documents...
Step 3: Zeroing out 20.0% least important dimensions...
Step 4: Performing new retrieval with modified query...

3. LLM RERANK:
Step 1: Initial retrieval (top-1000)...
Step 2: Generating and emb

## 4. Analysis and Metrics

Let's analyze the effectiveness of each approach by looking at score differences and result changes.

In [ ]:
def analyze_approach_effectiveness(results):
    """
    Analyze the effectiveness of each approach
    """
    print(f"\n{'='*80}")
    print(f"APPROACH EFFECTIVENESS ANALYSIS")
    print(f"{'='*80}")
    
    query = list(results.values())[0]['query']
    baseline_results = results['prf_rerank']['original_results']
    baseline_ids = [r['product_id'] for r in baseline_results[:5]]
    
    print(f"\nQuery: '{query}'")
    print(f"Baseline top-5 IDs: {baseline_ids}")
    
    for method_name, method_results in results.items():
        print(f"\n{method_name.upper().replace('_', ' ')}:")
        print("-" * 40)
        
        # Get final results
        if 'reranked_results' in method_results:
            final_results = method_results['reranked_results']
            score_key = 'new_score'
        else:
            final_results = method_results['refetched_results']
            score_key = 'score'
        
        final_ids = [r['product_id'] for r in final_results[:5]]
        
        # Calculate metrics
        overlap = len(set(baseline_ids) & set(final_ids))
        changes = 5 - overlap
        avg_score = np.mean([r[score_key] for r in final_results[:5]])
        
        print(f"  • Top-5 IDs: {final_ids}")
        print(f"  • Overlap with baseline: {overlap}/5")
        print(f"  • Changes from baseline: {changes}/5")
        print(f"  • Average score: {avg_score:.4f}")
        
        # For rerank methods, show score improvements
        if 'reranked_results' in method_results:
            original_scores = [r['original_score'] for r in final_results[:5]]
            new_scores = [r['new_score'] for r in final_results[:5]]
            score_changes = [new - orig for new, orig in zip(new_scores, original_scores)]
            print(f"  • Score changes: {[f'{c:+.4f}' for c in score_changes]}")

def compare_query_embeddings(results):
    """
    Compare how different approaches modify the query embeddings
    """
    print(f"\n{'='*80}")
    print(f"QUERY EMBEDDING MODIFICATIONS")
    print(f"{'='*80}")
    
    query = list(results.values())[0]['query']
    original_embedding = results['prf_rerank']['original_query_embedding']
    
    print(f"\nQuery: '{query}'")
    print(f"Original embedding norm: {np.linalg.norm(original_embedding):.4f}")
    print(f"Original embedding first 10 dims: {original_embedding[:10]}")
    
    for method_name, method_results in results.items():
        modified_embedding = method_results['modified_query_embedding']
        
        # Calculate metrics
        norm_diff = np.linalg.norm(modified_embedding) - np.linalg.norm(original_embedding)
        cosine_sim = np.dot(original_embedding, modified_embedding) / \
                    (np.linalg.norm(original_embedding) * np.linalg.norm(modified_embedding))
        zero_dims = np.sum(modified_embedding == 0)
        
        print(f"\n{method_name.upper().replace('_', ' ')}:")
        print(f"  • Modified norm: {np.linalg.norm(modified_embedding):.4f} (Δ: {norm_diff:+.4f})")
        print(f"  • Cosine similarity with original: {cosine_sim:.4f}")
        print(f"  • Zeroed dimensions: {zero_dims}/{len(modified_embedding)}")
        print(f"  • Modified first 10 dims: {modified_embedding[:10]}")

# Analyze results for the first query
if all_results:
    first_query = list(all_results.keys())[0]
    analyze_approach_effectiveness(all_results[first_query])
    compare_query_embeddings(all_results[first_query])
else:
    print("Run the previous cell first to generate results!")

## 5. Parameter Experimentation

Let's experiment with different parameters to understand their impact on retrieval performance.

In [ ]:
def experiment_zero_out_ratios(query="black dress shirt"):
    """
    Experiment with different zero-out ratios
    """
    print(f"\n{'='*80}")
    print(f"ZERO-OUT RATIO EXPERIMENTATION")
    print(f"{'='*80}")
    
    ratios = [0.0, 0.1, 0.2, 0.3, 0.5]
    
    print(f"\nQuery: '{query}'")
    
    for ratio in ratios:
        print(f"\nZero-out ratio: {ratio} ({ratio*100}% dimensions zeroed)")
        print("-" * 50)
        
        # Test PRF rerank
        prf_result = prf_dime.prf_rerank(query, zero_out_ratio=ratio, final_top_k=3)
        prf_avg_score = np.mean([r['new_score'] for r in prf_result['reranked_results']])
        
        # Test LLM rerank  
        llm_result = llm_dime.llm_rerank(query, zero_out_ratio=ratio, final_top_k=3)
        llm_avg_score = np.mean([r['new_score'] for r in llm_result['reranked_results']])
        
        # Test Magnitude rerank
        magnitude_result = magnitude_dime.magnitude_rerank(query, zero_out_ratio=ratio, final_top_k=3)
        magnitude_avg_score = np.mean([r['new_score'] for r in magnitude_result['reranked_results']])
        
        print(f"PRF Rerank avg score: {prf_avg_score:.4f}")
        print(f"LLM Rerank avg score: {llm_avg_score:.4f}")
        print(f"Magnitude Rerank avg score: {magnitude_avg_score:.4f}")

def experiment_prf_k_values(query="black dress shirt"):
    """
    Experiment with different PRF-k values
    """
    print(f"\n{'='*80}")
    print(f"PRF-K VALUE EXPERIMENTATION")
    print(f"{'='*80}")
    
    prf_k_values = [3, 5, 10, 20]
    
    print(f"\nQuery: '{query}'")
    
    for k in prf_k_values:
        print(f"\nPRF-k: {k}")
        print("-" * 30)
        
        # Test both rerank and refetch
        rerank_result = prf_dime.prf_rerank(query, prf_k=k, final_top_k=3)
        refetch_result = prf_dime.prf_refetch(query, prf_k=k, final_top_k=3)
        
        rerank_avg = np.mean([r['new_score'] for r in rerank_result['reranked_results']])
        refetch_avg = np.mean([r['score'] for r in refetch_result['refetched_results']])
        
        print(f"PRF Rerank avg score: {rerank_avg:.4f}")
        print(f"PRF Refetch avg score: {refetch_avg:.4f}")

def experiment_attention_types(query="black dress shirt"):
    """
    Experiment with different attention types for PRF
    """
    print(f"\n{'='*80}")
    print(f"ATTENTION TYPE EXPERIMENTATION")
    print(f"{'='*80}")
    
    attention_types = ["linear", "softmax"]
    temperatures = [0.5, 1.0, 2.0]
    
    print(f"\nQuery: '{query}'")
    
    for att_type in attention_types:
        print(f"\nAttention type: {att_type}")
        print("-" * 40)
        
        if att_type == "linear":
            # Test unweighted vs weighted
            for weighted in [False, True]:
                result = prf_dime.prf_rerank(query, weighted=weighted, attention_type=att_type, final_top_k=3)
                avg_score = np.mean([r['new_score'] for r in result['reranked_results']])
                print(f"  Weighted={weighted}: {avg_score:.4f}")
        else:
            # Test different temperatures for softmax
            for temp in temperatures:
                result = prf_dime.prf_rerank(query, weighted=True, attention_type=att_type, 
                                           temperature=temp, final_top_k=3)
                avg_score = np.mean([r['new_score'] for r in result['reranked_results']])
                print(f"  Temperature={temp}: {avg_score:.4f}")

# Run experiments
experiment_zero_out_ratios()
experiment_prf_k_values() 
experiment_attention_types()

## 6. Conclusion and Next Steps

This notebook implements the core DIME approaches:

### Implemented Approaches:
1. **PRF-based Rerank**: Uses PRF to compute centroids and re-scores existing results
2. **PRF-based Refetch**: Uses PRF to compute centroids and performs new retrieval
3. **LLM-based Rerank**: Uses LLM-generated docs to determine importance and re-scores
4. **LLM-based Refetch**: Uses LLM-generated docs to determine importance and performs new retrieval
5. **Magnitude-based Rerank**: Uses query magnitude to determine importance and re-scores
6. **Magnitude-based Refetch**: Uses query magnitude to determine importance and performs new retrieval

### Key Components:
- **Dimension Importance Estimation**: 
  - PRF: Element-wise product between centroid and query
  - LLM: Element-wise product between LLM-doc embedding and query  
  - Magnitude: Absolute values of query dimensions |qi|
- **Dimension Zeroing**: Zeros out least important dimensions based on importance scores
- **Flexible Parameters**: Supports different zero-out ratios, PRF-k values, attention types

### Next Steps:
1. **Real LLM Integration**: Replace simulated LLM docs with actual LLM-generated content
2. **Evaluation Metrics**: Add proper evaluation metrics (NDCG, MAP, etc.)
3. **Hyperparameter Tuning**: Systematic optimization of parameters
4. **Dataset Expansion**: Test on larger, more diverse datasets
5. **Comparison with SOTA**: Compare against other dense retrieval methods

The implementation is modular and extensible, making it easy to integrate the actual DIME methodology when ready.